# **1 - Un exemple**

Alfred, Berthe, Camille et Daphné étudient ensemble à Toulouse et veulent co voiturer pour les vacances de Noel. Alfred est à Narbonne, Berthe à Vienne, Camille à Lyon et Daphné à Munich. Les distances à Toulouse des villes d'arrivée sont les suivantes : T A = 100 km, T B = 450
km, T C = 500 et T D = 1450 km

Proposer et discuter une solution de partage équitable du cout du covoiturage.

Pour obtenir une solution de partage équitable, on peut utiliser la solution de Shapley.

Ainsi, on calul :


_           |  $\Delta A$  |  $\Delta B$  |  $\Delta C$  |  $\Delta D$  |
------------|------|------|------|------|
A B C D     | 100  | 350  | 50   | 950  |
A B D C     | 100  | 350  | 0    | 1000 |
A C B D     | 100  | 0    | 400  | 950  |
A C D B     | 100  | 0    | 400  | 950  |
A D C B     | 100  | 0    | 0    | 1350 |
A D B C     | 100  | 0    | 0    | 1350 |
B A C D     | 0    | 450  | 50   | 950  |
B A D C     | 0    | 450  | 0    | 1000 |
B C A D     | 0    | 450  | 50   | 950  |
B C D A     | 0    | 450  | 50   | 950  |
B D A C     | 0    | 450  | 0    | 1000 |
B D C A     | 0    | 450  | 0    | 1000 |
C A B D     | 0    | 0    | 500  | 950  |
C A D B     | 0    | 0    | 500  | 950  |
C B A D     | 0    | 0    | 500  | 950  |
C B D A     | 0    | 0    | 500  | 950  |
C D A B     | 0    | 0    | 500  | 950  |
C D B A     | 0    | 0    | 500  | 950  |
D A C B     | 0    | 0    | 0    | 1450 |
D A B C     | 0    | 0    | 0    | 1450 |
D B A C     | 0    | 0    | 0    | 1450 |
D B C A     | 0    | 0    | 0    | 1450 |
D C A B     | 0    | 0    | 0    | 1450 |
D C B A     | 0    | 0    | 0    | 1450 |


Et donc on obtient:

<!-- $\textrm{Texte}_\textrm{Indice}$ -->

$ x_{A} = \frac{100*6}{24} = 25 $

$ x_{B} = \frac{350*2+450*6}{24} = \frac{425}{3} \approx 141.67 $

$ x_{C} = \frac{50*4+400*2+500*6}{24} = \frac{500}{3} \approx 166.67 $

$ x_{D} = \frac{950*12+100*4+1350*2+1450*6}{24} \approx 1116.67 $



Cette solution vérifie bien le stand-alone test et est bien dans le noyau.
Donc aucun voyageurs n'ont intêret à voyager seuls (ou en sous-groupe) sans covoiturer avec le reste du groupe.

# **2 - Un logiciel**

In [2]:
# Definition des parametres

trajet_predefini = "Toulouse-Munich"

distances = {
    ("Berthe", "Toulouse-Vienne"):      450,
    ("Alfred", "Toulouse-Narbonne"):    100,
    ("Camille", "Toulouse-Lyon"):       500,
    ("Daphne", "Toulouse-Munich"):      1450
    }

cout_km = 1

In [3]:
# Fonction qui permet de calculer les coûts de tuples de voyageurs si ils ne covoiturent pas avec le groupe
def cout_tuple(list_cout_seul, nb):
    cout = {}
    nbPers = len(list_cout_seul)
    if nb >= 2:
        for j in range(nbPers):
            for k in range(j+1, nbPers):
                if nb >= 3:
                    for l in range(k+1, nbPers):
                        if nb>=4:
                            if nb>=5:
                                print("error nb > 4 !")
                            for m in range(l+1, nbPers):
                                p1, c1 = list_cout_seul[j]
                                p2, c2 = list_cout_seul[k]
                                p3, c3 = list_cout_seul[l]
                                p4, c4 = list_cout_seul[m]
                                cout[(p1, p2, p3, p4)] = max(c1, c2, c3, c4)
                        else:
                            p1, c1 = list_cout_seul[j]
                            p2, c2 = list_cout_seul[k]
                            p3, c3 = list_cout_seul[l]
                            cout[(p1, p2, p3)] = max(c1, c2, c3)
                else:
                    p1, c1 = list_cout_seul[j]
                    p2, c2 = list_cout_seul[k]
                    cout[(p1, p2)] = max(c1, c2)
    else:
        print("error nb < 2 !")
    return cout

In [4]:
# Fonction qui teste une allocation donnée, dit si cette allocation satisfait le stand-alone test, calcule les couts si seuls
# de tous les sous-groupes de passagers et dit si cette allocation est dans le noyau

def test_alloc(distances, cout_km, allocation):
    distances = dict(sorted(distances.items(), key=lambda x:x[1]))

    dict_cout_seul = {}
    for elt, c in distances.items():
        p, t = elt
        dict_cout_seul[p] = c * cout_km

    nbPers = len(distances)

    list_cout_seul = list(dict_cout_seul.items())

    for i in range(2, nbPers):
        print("i=", i, " cout_tuple:", cout_tuple(list_cout_seul,i))

    # Stand alone test
    b = True
    for i in range(nbPers):
        p, c = list_cout_seul[i]
        if allocation[p] > c:
            b = False
            print("Cette allocation ne respecte pas le stand alone test car pour ", p, " son allocation vaut : ", allocation[p], " mais tout seul son cout serait : ", c)
    if b:
        print("Cette allocation respecte le stand alone test")


    # Noyau
    if not b:
        print("Cette allocation n'est pas dans le noyau car elle ne respecte pas le stand alone test")
    else:
        for i in range(2, nbPers+1):
            cout_tuplee = cout_tuple(list_cout_seul, i)
            for elt, cout_seul in cout_tuplee.items():
                cout_alloc = 0
                for j in range(len(elt)):
                    cout_alloc += allocation[elt[j]]
                if cout_alloc > cout_seul:
                    print("Cette allocation n'est pas dans le noyau car pour le couple ", elt, " le cout seul vaut : ", cout_seul, " et le cout en collaborant vaut : ", cout_alloc)
                    b = False
        if b:
            print("Cette allocation est dans le noyau")


In [5]:
# Definition du logiciel, qui en fonction du trajet predefini, des distances et du cout par km
# renvoie le cout total, le cout si seul pour chaque passager, l'allocation à couts proportionnels et l'allocation par séparation
# et teste ces allocations

def logiciel(trajet_predefini, distances, cout_km):
    distances = dict(sorted(distances.items(), key=lambda x:x[1]))

    dict_cout_seul = {}
    for elt, c in distances.items():
        p, t = elt
        dict_cout_seul[p] = c * cout_km
        if t == trajet_predefini:
            cout_total = c * cout_km


    # Allocation à couts proportionnels
    l = []
    cpt = 0
    for p, c in dict_cout_seul.items():
        if cpt==0:
            cpt+=1
            name = p
        else:
            l.append(c/dict_cout_seul[name])

    alloc_cout_proportionnels={}
    alloc_cout_proportionnels[name] = cout_total / (sum(l)+ 1)
    cpt = 0
    for p, c in dict_cout_seul.items():
        if cpt==0:
            cpt+=1
        else:
            alloc_cout_proportionnels[p] = (c * alloc_cout_proportionnels[name]) / dict_cout_seul[name]


    # Allocation par separation
    alloc_separation = {}
    nbPers = len(distances)
    cout_del = 0
    cout_add = 0
    for p, c in dict_cout_seul.items():
        alloc_separation[p] = (c - cout_del)/nbPers + cout_add
        cout_add += (c - cout_del)/nbPers
        nbPers-=1
        cout_del = c


    print("Cout total:", cout_total)
    print("Cout seul:", dict_cout_seul)
    print("\nAllocation a couts proportionnels:", alloc_cout_proportionnels)
    print("Allocation par separation:", alloc_separation)


    print("\nTest de l'allocation a couts proportionnels")
    test_alloc(distances, cout_km, alloc_cout_proportionnels)

    print("\n\n")
    print("Test de l'allocation par separation")
    test_alloc(distances, cout_km, alloc_separation)

In [6]:
logiciel(trajet_predefini, distances, cout_km)

Cout total: 1450
Cout seul: {'Alfred': 100, 'Berthe': 450, 'Camille': 500, 'Daphne': 1450}

Allocation a couts proportionnels: {'Alfred': 58.0, 'Berthe': 261.0, 'Camille': 290.0, 'Daphne': 841.0}
Allocation par separation: {'Alfred': 25.0, 'Berthe': 141.66666666666669, 'Camille': 166.66666666666669, 'Daphne': 1116.6666666666667}

Test de l'allocation a couts proportionnels
i= 2  cout_tuple: {('Alfred', 'Berthe'): 450, ('Alfred', 'Camille'): 500, ('Alfred', 'Daphne'): 1450, ('Berthe', 'Camille'): 500, ('Berthe', 'Daphne'): 1450, ('Camille', 'Daphne'): 1450}
i= 3  cout_tuple: {('Alfred', 'Berthe', 'Camille'): 500, ('Alfred', 'Berthe', 'Daphne'): 1450, ('Alfred', 'Camille', 'Daphne'): 1450, ('Berthe', 'Camille', 'Daphne'): 1450}
Cette allocation respecte le stand alone test
Cette allocation n'est pas dans le noyau car pour le couple  ('Berthe', 'Camille')  le cout seul vaut :  500  et le cout en collaborant vaut :  551.0
Cette allocation n'est pas dans le noyau car pour le couple  ('Alfre